In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os

In [29]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [30]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [31]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [4]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [36]:
net = ResNet18()

In [37]:
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

In [39]:
# Training
def train(epoch):
    print('\nTrain Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))



In [40]:
def test(epoch):
    print('\nTest Epoch: %d' % epoch)
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
            
            # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


In [41]:
for epoch in range(start_epoch, start_epoch+10):
    train(epoch)
    test(epoch)


Train Epoch: 0
Loss: 2.427 | Acc: 3.125% (4/128)
Loss: 2.760 | Acc: 7.812% (20/256)
Loss: 3.075 | Acc: 9.115% (35/384)
Loss: 3.431 | Acc: 10.547% (54/512)
Loss: 3.811 | Acc: 10.625% (68/640)
Loss: 3.864 | Acc: 11.719% (90/768)
Loss: 3.738 | Acc: 12.054% (108/896)
Loss: 3.929 | Acc: 12.012% (123/1024)
Loss: 3.911 | Acc: 12.240% (141/1152)
Loss: 3.887 | Acc: 11.953% (153/1280)
Loss: 3.888 | Acc: 12.997% (183/1408)
Loss: 3.837 | Acc: 12.695% (195/1536)
Loss: 3.790 | Acc: 12.560% (209/1664)
Loss: 3.760 | Acc: 12.891% (231/1792)
Loss: 3.726 | Acc: 12.917% (248/1920)
Loss: 3.688 | Acc: 12.842% (263/2048)
Loss: 3.637 | Acc: 13.051% (284/2176)
Loss: 3.593 | Acc: 13.108% (302/2304)
Loss: 3.543 | Acc: 13.199% (321/2432)
Loss: 3.504 | Acc: 13.477% (345/2560)
Loss: 3.480 | Acc: 13.430% (361/2688)
Loss: 3.456 | Acc: 13.459% (379/2816)
Loss: 3.447 | Acc: 13.553% (399/2944)
Loss: 3.406 | Acc: 13.477% (414/3072)
Loss: 3.386 | Acc: 13.469% (431/3200)
Loss: 3.348 | Acc: 13.522% (450/3328)
Loss: 3.324 |

Loss: 2.156 | Acc: 25.606% (6850/26752)
Loss: 2.154 | Acc: 25.625% (6888/26880)
Loss: 2.152 | Acc: 25.644% (6926/27008)
Loss: 2.151 | Acc: 25.634% (6956/27136)
Loss: 2.149 | Acc: 25.682% (7002/27264)
Loss: 2.148 | Acc: 25.683% (7035/27392)
Loss: 2.146 | Acc: 25.687% (7069/27520)
Loss: 2.145 | Acc: 25.720% (7111/27648)
Loss: 2.143 | Acc: 25.760% (7155/27776)
Loss: 2.141 | Acc: 25.796% (7198/27904)
Loss: 2.139 | Acc: 25.853% (7247/28032)
Loss: 2.138 | Acc: 25.902% (7294/28160)
Loss: 2.136 | Acc: 25.930% (7335/28288)
Loss: 2.134 | Acc: 25.996% (7387/28416)
Loss: 2.132 | Acc: 26.026% (7429/28544)
Loss: 2.130 | Acc: 26.081% (7478/28672)
Loss: 2.128 | Acc: 26.149% (7531/28800)
Loss: 2.127 | Acc: 26.179% (7573/28928)
Loss: 2.125 | Acc: 26.194% (7611/29056)
Loss: 2.124 | Acc: 26.216% (7651/29184)
Loss: 2.123 | Acc: 26.262% (7698/29312)
Loss: 2.121 | Acc: 26.315% (7747/29440)
Loss: 2.119 | Acc: 26.336% (7787/29568)
Loss: 2.118 | Acc: 26.360% (7828/29696)
Loss: 2.116 | Acc: 26.378% (7867/29824)


Loss: 1.540 | Acc: 43.760% (1094/2500)
Loss: 1.550 | Acc: 43.115% (1121/2600)
Loss: 1.550 | Acc: 43.074% (1163/2700)
Loss: 1.551 | Acc: 42.929% (1202/2800)
Loss: 1.557 | Acc: 42.931% (1245/2900)
Loss: 1.553 | Acc: 42.767% (1283/3000)
Loss: 1.550 | Acc: 42.935% (1331/3100)
Loss: 1.549 | Acc: 43.031% (1377/3200)
Loss: 1.548 | Acc: 43.121% (1423/3300)
Loss: 1.549 | Acc: 43.118% (1466/3400)
Loss: 1.550 | Acc: 43.143% (1510/3500)
Loss: 1.547 | Acc: 43.056% (1550/3600)
Loss: 1.548 | Acc: 43.054% (1593/3700)
Loss: 1.548 | Acc: 43.026% (1635/3800)
Loss: 1.545 | Acc: 43.154% (1683/3900)
Loss: 1.546 | Acc: 43.175% (1727/4000)
Loss: 1.545 | Acc: 43.268% (1774/4100)
Loss: 1.545 | Acc: 43.214% (1815/4200)
Loss: 1.544 | Acc: 43.163% (1856/4300)
Loss: 1.542 | Acc: 43.182% (1900/4400)
Loss: 1.539 | Acc: 43.311% (1949/4500)
Loss: 1.538 | Acc: 43.239% (1989/4600)
Loss: 1.535 | Acc: 43.319% (2036/4700)
Loss: 1.534 | Acc: 43.479% (2087/4800)
Loss: 1.534 | Acc: 43.449% (2129/4900)
Loss: 1.532 | Acc: 43.560

Loss: 1.554 | Acc: 42.367% (7321/17280)
Loss: 1.554 | Acc: 42.360% (7374/17408)
Loss: 1.554 | Acc: 42.376% (7431/17536)
Loss: 1.553 | Acc: 42.408% (7491/17664)
Loss: 1.553 | Acc: 42.395% (7543/17792)
Loss: 1.553 | Acc: 42.394% (7597/17920)
Loss: 1.553 | Acc: 42.387% (7650/18048)
Loss: 1.552 | Acc: 42.430% (7712/18176)
Loss: 1.552 | Acc: 42.411% (7763/18304)
Loss: 1.552 | Acc: 42.405% (7816/18432)
Loss: 1.552 | Acc: 42.446% (7878/18560)
Loss: 1.550 | Acc: 42.514% (7945/18688)
Loss: 1.550 | Acc: 42.501% (7997/18816)
Loss: 1.551 | Acc: 42.478% (8047/18944)
Loss: 1.551 | Acc: 42.455% (8097/19072)
Loss: 1.551 | Acc: 42.443% (8149/19200)
Loss: 1.551 | Acc: 42.441% (8203/19328)
Loss: 1.552 | Acc: 42.439% (8257/19456)
Loss: 1.551 | Acc: 42.453% (8314/19584)
Loss: 1.551 | Acc: 42.482% (8374/19712)
Loss: 1.551 | Acc: 42.500% (8432/19840)
Loss: 1.551 | Acc: 42.513% (8489/19968)
Loss: 1.550 | Acc: 42.516% (8544/20096)
Loss: 1.549 | Acc: 42.539% (8603/20224)
Loss: 1.549 | Acc: 42.527% (8655/20352)


Loss: 1.486 | Acc: 45.250% (19519/43136)
Loss: 1.486 | Acc: 45.269% (19585/43264)
Loss: 1.486 | Acc: 45.278% (19647/43392)
Loss: 1.486 | Acc: 45.283% (19707/43520)
Loss: 1.485 | Acc: 45.322% (19782/43648)
Loss: 1.484 | Acc: 45.340% (19848/43776)
Loss: 1.484 | Acc: 45.356% (19913/43904)
Loss: 1.483 | Acc: 45.365% (19975/44032)
Loss: 1.483 | Acc: 45.383% (20041/44160)
Loss: 1.483 | Acc: 45.385% (20100/44288)
Loss: 1.482 | Acc: 45.391% (20161/44416)
Loss: 1.482 | Acc: 45.414% (20229/44544)
Loss: 1.482 | Acc: 45.433% (20296/44672)
Loss: 1.481 | Acc: 45.449% (20361/44800)
Loss: 1.480 | Acc: 45.473% (20430/44928)
Loss: 1.480 | Acc: 45.479% (20491/45056)
Loss: 1.479 | Acc: 45.492% (20555/45184)
Loss: 1.479 | Acc: 45.516% (20624/45312)
Loss: 1.479 | Acc: 45.506% (20678/45440)
Loss: 1.479 | Acc: 45.521% (20743/45568)
Loss: 1.479 | Acc: 45.531% (20806/45696)
Loss: 1.478 | Acc: 45.550% (20873/45824)
Loss: 1.478 | Acc: 45.548% (20930/45952)
Loss: 1.478 | Acc: 45.556% (20992/46080)
Loss: 1.478 | Ac

...

Loss: 0.692 | Acc: 76.691% (6212/8100)
Loss: 0.691 | Acc: 76.683% (6288/8200)
Loss: 0.692 | Acc: 76.627% (6360/8300)
Loss: 0.693 | Acc: 76.560% (6431/8400)
Loss: 0.693 | Acc: 76.565% (6508/8500)
Loss: 0.694 | Acc: 76.512% (6580/8600)
Loss: 0.694 | Acc: 76.460% (6652/8700)
Loss: 0.695 | Acc: 76.455% (6728/8800)
Loss: 0.695 | Acc: 76.449% (6804/8900)
Loss: 0.696 | Acc: 76.444% (6880/9000)
Loss: 0.696 | Acc: 76.451% (6957/9100)
Loss: 0.694 | Acc: 76.533% (7041/9200)
Loss: 0.697 | Acc: 76.419% (7107/9300)
Loss: 0.697 | Acc: 76.372% (7179/9400)
Loss: 0.697 | Acc: 76.379% (7256/9500)
Loss: 0.697 | Acc: 76.365% (7331/9600)
Loss: 0.696 | Acc: 76.412% (7412/9700)
Loss: 0.697 | Acc: 76.337% (7481/9800)
Loss: 0.698 | Acc: 76.293% (7553/9900)
Loss: 0.698 | Acc: 76.250% (7625/10000)

Train Epoch: 8
Loss: 0.472 | Acc: 83.594% (107/128)
Loss: 0.450 | Acc: 83.594% (214/256)
Loss: 0.492 | Acc: 81.510% (313/384)
Loss: 0.500 | Acc: 81.445% (417/512)
Loss: 0.523 | Acc: 80.469% (515/640)
Loss: 0.524 | Acc:

Loss: 0.585 | Acc: 79.880% (19120/23936)
Loss: 0.584 | Acc: 79.908% (19229/24064)
Loss: 0.584 | Acc: 79.919% (19334/24192)
Loss: 0.585 | Acc: 79.905% (19433/24320)
Loss: 0.585 | Acc: 79.896% (19533/24448)
Loss: 0.585 | Acc: 79.895% (19635/24576)
Loss: 0.584 | Acc: 79.906% (19740/24704)
Loss: 0.584 | Acc: 79.905% (19842/24832)
Loss: 0.584 | Acc: 79.904% (19944/24960)
Loss: 0.584 | Acc: 79.891% (20043/25088)
Loss: 0.584 | Acc: 79.886% (20144/25216)
Loss: 0.585 | Acc: 79.869% (20242/25344)
Loss: 0.585 | Acc: 79.868% (20344/25472)
Loss: 0.585 | Acc: 79.867% (20446/25600)
Loss: 0.586 | Acc: 79.839% (20541/25728)
Loss: 0.586 | Acc: 79.846% (20645/25856)
Loss: 0.585 | Acc: 79.861% (20751/25984)
Loss: 0.585 | Acc: 79.848% (20850/26112)
Loss: 0.586 | Acc: 79.851% (20953/26240)
Loss: 0.586 | Acc: 79.839% (21052/26368)
Loss: 0.585 | Acc: 79.842% (21155/26496)
Loss: 0.586 | Acc: 79.826% (21253/26624)
Loss: 0.586 | Acc: 79.822% (21354/26752)
Loss: 0.586 | Acc: 79.792% (21448/26880)
Loss: 0.586 | Ac

Loss: 0.584 | Acc: 79.932% (39595/49536)
Loss: 0.585 | Acc: 79.925% (39694/49664)
Loss: 0.585 | Acc: 79.924% (39796/49792)
Loss: 0.584 | Acc: 79.930% (39901/49920)
Loss: 0.584 | Acc: 79.938% (39969/50000)

Test Epoch: 8
Loss: 0.721 | Acc: 74.000% (74/100)
Loss: 0.658 | Acc: 75.500% (151/200)
Loss: 0.645 | Acc: 76.333% (229/300)
Loss: 0.632 | Acc: 77.500% (310/400)
Loss: 0.632 | Acc: 77.200% (386/500)
Loss: 0.601 | Acc: 77.500% (465/600)
Loss: 0.624 | Acc: 76.429% (535/700)
Loss: 0.676 | Acc: 75.125% (601/800)
Loss: 0.689 | Acc: 75.444% (679/900)
Loss: 0.674 | Acc: 76.000% (760/1000)
Loss: 0.672 | Acc: 76.091% (837/1100)
Loss: 0.672 | Acc: 75.917% (911/1200)
Loss: 0.667 | Acc: 76.077% (989/1300)
Loss: 0.664 | Acc: 76.286% (1068/1400)
Loss: 0.659 | Acc: 76.400% (1146/1500)
Loss: 0.662 | Acc: 76.438% (1223/1600)
Loss: 0.660 | Acc: 76.765% (1305/1700)
Loss: 0.665 | Acc: 76.556% (1378/1800)
Loss: 0.668 | Acc: 76.632% (1456/1900)
Loss: 0.676 | Acc: 76.300% (1526/2000)
Loss: 0.679 | Acc: 76.1

Loss: 0.542 | Acc: 80.980% (11091/13696)
Loss: 0.542 | Acc: 81.004% (11198/13824)
Loss: 0.541 | Acc: 81.021% (11304/13952)
Loss: 0.542 | Acc: 81.009% (11406/14080)
Loss: 0.542 | Acc: 81.011% (11510/14208)
Loss: 0.541 | Acc: 81.027% (11616/14336)
Loss: 0.541 | Acc: 81.029% (11720/14464)
Loss: 0.542 | Acc: 81.010% (11821/14592)
Loss: 0.542 | Acc: 81.012% (11925/14720)
Loss: 0.541 | Acc: 81.008% (12028/14848)
Loss: 0.542 | Acc: 80.976% (12127/14976)
Loss: 0.542 | Acc: 80.959% (12228/15104)
Loss: 0.542 | Acc: 80.948% (12330/15232)
Loss: 0.542 | Acc: 80.983% (12439/15360)
Loss: 0.542 | Acc: 80.992% (12544/15488)
Loss: 0.542 | Acc: 80.975% (12645/15616)
Loss: 0.542 | Acc: 80.971% (12748/15744)
Loss: 0.543 | Acc: 80.916% (12843/15872)
Loss: 0.544 | Acc: 80.894% (12943/16000)
Loss: 0.545 | Acc: 80.872% (13043/16128)
Loss: 0.545 | Acc: 80.893% (13150/16256)
Loss: 0.544 | Acc: 80.908% (13256/16384)
Loss: 0.545 | Acc: 80.862% (13352/16512)
Loss: 0.546 | Acc: 80.811% (13447/16640)
Loss: 0.545 | Ac

Loss: 0.554 | Acc: 80.850% (31771/39296)
Loss: 0.554 | Acc: 80.849% (31874/39424)
Loss: 0.555 | Acc: 80.848% (31977/39552)
Loss: 0.555 | Acc: 80.854% (32083/39680)
Loss: 0.555 | Acc: 80.861% (32189/39808)
Loss: 0.555 | Acc: 80.872% (32297/39936)
Loss: 0.555 | Acc: 80.878% (32403/40064)
Loss: 0.555 | Acc: 80.872% (32504/40192)
Loss: 0.554 | Acc: 80.883% (32612/40320)
Loss: 0.554 | Acc: 80.892% (32719/40448)
Loss: 0.554 | Acc: 80.895% (32824/40576)
Loss: 0.554 | Acc: 80.909% (32933/40704)
Loss: 0.553 | Acc: 80.919% (33041/40832)
Loss: 0.553 | Acc: 80.930% (33149/40960)
Loss: 0.553 | Acc: 80.938% (33256/41088)
Loss: 0.553 | Acc: 80.952% (33365/41216)
Loss: 0.553 | Acc: 80.960% (33472/41344)
Loss: 0.553 | Acc: 80.949% (33571/41472)
Loss: 0.553 | Acc: 80.950% (33675/41600)
Loss: 0.553 | Acc: 80.953% (33780/41728)
Loss: 0.553 | Acc: 80.954% (33884/41856)
Loss: 0.553 | Acc: 80.964% (33992/41984)
Loss: 0.553 | Acc: 80.951% (34090/42112)
Loss: 0.553 | Acc: 80.956% (34196/42240)
Loss: 0.553 | Ac

In [42]:
print(best_acc)

76.89


# Vanilla ResNet18

In [18]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
from torchvision import models
res_net = models.resnet50(pretrained=True)
#for param in res_net.parameters():
    #param.requires_grad = False
num_ftrs = res_net.fc.in_features
res_net.fc = nn.Linear(num_ftrs, len(classes))

res_net = res_net.to(device)

criterion = nn.CrossEntropyLoss().to(device)

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(res_net.fc.parameters(), lr=0.01, momentum=0.9, weight_decay = 1e-4)

from torch.optim import lr_scheduler
# Decay LR by a factor of 0.1 every 20 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=20, gamma=0.1)

In [20]:
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

'''
data_dir = 'data/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
'''
dataloaders = {'train': trainloader, 'val': testloader}
dataset_sizes = {'train': len(trainset), 'val': len(testset)}

In [21]:


def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [22]:
model_conv = train_model(res_net, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
train Loss: 1.1761 Acc: 0.6894
val Loss: 1.0712 Acc: 0.7396

Epoch 1/19
----------
train Loss: 1.0553 Acc: 0.7313
val Loss: 1.1787 Acc: 0.7201

Epoch 2/19
----------
train Loss: 1.0392 Acc: 0.7381
val Loss: 0.8413 Acc: 0.7867

Epoch 3/19
----------
train Loss: 1.0118 Acc: 0.7440
val Loss: 0.8195 Acc: 0.7842

Epoch 4/19
----------
train Loss: 0.9916 Acc: 0.7489
val Loss: 0.8519 Acc: 0.7770

Epoch 5/19
----------
train Loss: 0.9643 Acc: 0.7528
val Loss: 0.7477 Acc: 0.7944

Epoch 6/19
----------
train Loss: 0.9643 Acc: 0.7548
val Loss: 1.2058 Acc: 0.7299

Epoch 7/19
----------
train Loss: 0.9360 Acc: 0.7623
val Loss: 0.7980 Acc: 0.7929

Epoch 8/19
----------
train Loss: 0.9115 Acc: 0.7648
val Loss: 0.7101 Acc: 0.8032

Epoch 9/19
----------
train Loss: 0.9006 Acc: 0.7658
val Loss: 0.7995 Acc: 0.7906

Epoch 10/19
----------
train Loss: 0.8844 Acc: 0.7683
val Loss: 0.7553 Acc: 0.7923

Epoch 11/19
----------
train Loss: 0.8723 Acc: 0.7701
val Loss: 0.8466 Acc: 0.7814

Ep